# [Development of a software module for recognizing the fingerspelling of the Russian Sign Language based on LSTM](https://www.researchgate.net/publication/355402809_Development_of_a_software_module_for_recognizing_the_fingerspelling_of_the_Russian_Sign_Language_based_on_LSTM)


In [ ]:
import numpy as np 
import pandas as pd 
import glob
import os
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import keras

## 1. Prepare data

In [ ]:
DATA_PATH = "/Users/kamilla/Desktop/masters_diploma/SOTA_MODELS/hands_SIBI_training-6.csv"
RANDOM_STATE = 1228
keras.utils.set_random_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

In [ ]:
df = pd.read_csv(DATA_PATH, header=0)
df.drop(0, axis=0, inplace=True) # первая строчка была проверочная, поэтому избавляемся от нее
df = df.sort_values(by=["class_type"])
df.reset_index(inplace=True, drop=True)

In [ ]:
df

In [ ]:
df['class_type'] = pd.Categorical(df["class_type"])
df['class_type'] = df.class_type.cat.codes

df.drop(['wristX', 'wristY', 'wristZ'], inplace=True, axis=1)

In [ ]:
df

In [ ]:
y_data = df.pop("class_type")
X_data = df.copy()
y_data = keras.utils.to_categorical(y_data)

In [ ]:
train_ratio = 0.90


x_train, x_val, y_train, y_val = train_test_split(X_data,
                                                    y_data, 
                                                    test_size= 1 - train_ratio,
                                                   shuffle=True,
                                                 random_state=RANDOM_STATE)

#test_ratio = 0.10
#validation_ratio = 0.10

#x_val, x_test, y_val, y_test = train_test_split(x_test, 
#                                                y_test, 
#                                                test_size=test_ratio/(test_ratio + validation_ratio)) 

x_train, x_val = np.array(x_train), np.array(x_val)

In [ ]:
print(f"X_train shape {x_train.shape}. y_train shape {y_train.shape}")
#print(f"X_test shape {x_test.shape}. y_test shape {y_test_cat.shape}")
print(f"X_val shape {x_val.shape}. y_val shape {y_val.shape}")

# Model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dropout, Dense

model_1 = Sequential()
model_1.add(Conv1D(15, 4, activation='relu', input_shape=(60,1)))
model_1.add(MaxPooling1D(pool_size=2))
model_1.add(Conv1D(30, 1,  activation='relu', padding="same", use_bias=True))
model_1.add(MaxPooling1D(pool_size=2))
model_1.add(Flatten())
model_1.add(Dropout(0.5))
model_1.add(Dense(25, activation='softmax'))

model_1.summary()

In [ ]:
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model_1.fit(
    x=x_train,
    y=y_train, 
    batch_size=64,
    epochs=200,
    verbose=1,
    validation_data=(x_val, y_val),
    validation_batch_size=32,
    callbacks=[callback]
)

In [ ]:
1

In [ ]:
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, 43+1)
#print(len(epochs))
plt.plot(epochs, loss_values, '-', label='Training loss', color='r')
plt.plot(epochs, val_loss_values, '-', label='Validation loss', color='g') 
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
y_val = np.argmax(y_val, axis=1).tolist()

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = model_1.predict(x_val)
#y_pred_val = model.predict(x_val_prep)

y_preds_class_1 = np.argmax(y_pred, axis=1).tolist()
#y_preds_class_val = np.argmax(y_pred_val, axis=1).tolist()

accuracy_test = accuracy_score(y_val, y_preds_class_1)
print(f"Точность на тесте (без аугументации): {accuracy_test}")

In [ ]:
from sklearn.metrics import classification_report


print(classification_report(y_val, y_preds_class_1)) 

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_val, y_preds_class_1, average='weighted')
recall = recall_score(y_val, y_preds_class_1, average='weighted')
f1 = f1_score(y_val, y_preds_class_1, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)